Create a .env file and add the following:

OPENAI_API_KEY="sk-..." [optional since you can also use Ollama]

EXCHANGE_RATE_API_KEY="..."

In [1]:
from dotenv import load_dotenv

import os
import requests
from typing import Type
from crewai.tools import BaseTool
from crewai import Agent, Task, Crew, Process

from pydantic import BaseModel, Field
load_dotenv()

True

In [2]:
from crewai import LLM 
llm = LLM(
    model="gpt-5-mini",
)

In [ ]:
# from crewai import LLM 

# llm = LLM(
#     model="ollama/llama3.2:1b",
#     base_url="http://localhost:11434"
# )

In [3]:
# Define the Currency Converter Tool
class CurrencyConverterInput(BaseModel):
    """Input schema for CurrencyConverterTool."""
    amount: float = Field(..., description="The amount to convert.")
    from_currency: str = Field(..., description="The source currency code (e.g., 'USD').")
    to_currency: str = Field(..., description="The target currency code (e.g., 'EUR').")

class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another using live exchange rates."
    args_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = os.getenv("EXCHANGE_RATE_API_KEY")  # Store API Key as an environment variable

    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        response = requests.get(url)
        if response.status_code != 200:
            return "Failed to fetch exchange rates."

        data = response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"

        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equivalent to {converted_amount:.2f} {to_currency}."


In [4]:
# query analyst agent that outputs a structured response

query_analyst = Agent(
    role="Query Analyst",
    goal="Analyze the user's query and output a structured response containing the amount, from_currency, and to_currency. The query is: '{query}'.",
    backstory=(
        "You are a language understanding expert with background in finance."
        "You understand the user's natural language query and output a structured response containing the amount, from_currency, and to_currency."
    ),
    verbose=True
)

query_task = Task(
    description="Understand the user's natural language query and extract the total amount, source currency, and target currency. The query is: '{query}'.",
    expected_output="A structured response to the user's query.",
    agent=query_analyst,
    output_pydantic=CurrencyConverterInput,
)

In [5]:

# Define the Agent
currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversions and financial insights.",
    backstory=(
        "You are a finance expert with deep knowledge of global exchange rates."
        "You help users with currency conversion and financial decision-making."
    ),
    tools=[CurrencyConverterTool()],
    verbose=True
)

# Define a Task
currency_conversion_task = Task(
    description=(
        "Accept the output from the query analyst agent and convert the amount using real-time exchange rates."
        "The input will contain the amount, from_currency, and to_currency."
        "Provide the equivalent amount and explain any relevant financial context."
    ),
    expected_output="A detailed response including the converted amount and financial insights.",
    agent=currency_analyst
)

In [10]:

# Form the Crew
crew = Crew(
    agents=[query_analyst, currency_analyst],
    tasks=[query_task, currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(inputs={"query": "How much is 100 USD in indian currency?"})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Query Analyst                                                                                           │
│                                                                                                                 │
│  Task: Understand the user's natural language query and extract the total amount, source currency, and target   │
│  currency. The query is: 'How much is 100 USD in indian currency?'.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Query Analyst                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "amount": 100,                                                                                               │
│    "from_currency": "USD",                                                                                      │
│    "to_currency": "INR"                                                                                         │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Currency Analyst                                                                                        │
│                                                                                                                 │
│  Task: Accept the output from the query analyst agent and convert the amount using real-time exchange           │
│  rates.The input will contain the amount, from_currency, and to_currency.Provide the equivalent amount and      │
│  explain any relevant financial context.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Currency Analyst                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to convert 100 USD to INR using the current exchange rate to provide an accurate and  │
│  detailed financial response.                                                                                   │
│                                                                                                                 │
│  Using Tool: Currency Converter Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "amount": 100,                                                                                               │
│    "from_currency": "USD",                                                                                      │
│    "to_currency": "INR"                                                                                         │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  100.0 USD is equivalent to 8945.39 INR.                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│  You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:          │
│                                                                                                                 │
│  Tool Name: Currency Converter Tool                                                                             │
│  Tool Arguments: {'amount': {'description': 'The amount to convert.', 'type': 'float'}, 'from_currency':        │
│  {'description': "The source currency code (e.g., 'USD').", 'type': 'str'}, 'to_currency': {'description':      │
│  "The target currency code (e.g., 'EUR').", 'type': 'str'}}                                                     │
│  Tool Description: Converts an amount from one currency to another using live exchange rates.                   │
│                                                                                                                 │
│  IMPORTANT: Use the following format in your response:                                                          │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, only one name of [Currency Converter Tool], just the name, exactly as it's         │
│  written.                                                                                                       │
│  Action Input: the input to the action, just a simple JSON object, enclosed in curly braces, using " to wrap    │
│  keys and values.                                                                                               │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│                                                                                                                 │
│  Once all necessary information is gathered, return the following format:                                       │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now know the final answer                                                                           │
│  Final Answer: the final answer to the original input question                                                  │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Currency Analyst                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  As of the current exchange rate, 100 US Dollars (USD) is equivalent to approximately 8945.39 Indian Rupees     │
│  (INR). This conversion reflects the live forex market rates where the USD generally holds strong purchasing    │
│  power compared to the INR. When converting currency, such as USD to INR, factors like inflation rates,         │
│  economic growth, interest rates, and geopolitical stability of both countries influence the exchange rate.     │
│  For individuals or businesses planning to convert USD to INR, understanding these rates helps in budgeting     │
│  for expenses, investments, or remittances more accurately. Keep in mind that currency values fluctuate         │
│  constantly due to market dynamics, so timely conversions or hedging strategies might be beneficial in          │
│  volatile periods.                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
from IPython.display import Markdown
Markdown(response.raw)

As of the current exchange rate, 100 US Dollars (USD) is equivalent to approximately 8945.39 Indian Rupees (INR). This conversion reflects the live forex market rates where the USD generally holds strong purchasing power compared to the INR. When converting currency, such as USD to INR, factors like inflation rates, economic growth, interest rates, and geopolitical stability of both countries influence the exchange rate. For individuals or businesses planning to convert USD to INR, understanding these rates helps in budgeting for expenses, investments, or remittances more accurately. Keep in mind that currency values fluctuate constantly due to market dynamics, so timely conversions or hedging strategies might be beneficial in volatile periods.